<a href="https://colab.research.google.com/github/INFINITY-RUBER/Curso_de_Tensorflow-2.0_Guia_completa_para_el_Nuevo_Tensorflow/blob/master/Collab_14_Entrenamiento_Distribuido.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Paso 1: Instalación de dependencias y configuración del entorno

In [ ]:
!pip install tensorflow-gpu==2.0.0.alpha0

## Paso 2: Importar las dependencias del proyecto

In [2]:
import time
import numpy as np
import tensorflow as tf

In [3]:
tf.__version__

'2.3.0'

## Paso 3: Pre procesado del dataset

### Cargar el dataset del MNIST

In [4]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

11493376/11490434 [==============================] - 0s 0us/step


### Normalización de las imágenes

In [5]:
X_train = X_train / 255.
X_test = X_test / 255.

In [6]:
X_train.shape

(60000, 28, 28)

### Redimensionar el dataset

In [7]:
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

In [8]:
X_train.shape

(60000, 784)

## Paso 4: Entrenamiento distribuido

### Definir un modelo normal (no distribuido)

In [9]:
model_normal = tf.keras.models.Sequential()

In [10]:
model_normal.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))

In [11]:
model_normal.add(tf.keras.layers.Dropout(rate=0.2))# para prevenir overfiting el 20% de las neuronas no se activaran

In [13]:
model_normal.add(tf.keras.layers.Dense(units=10, activation='softmax'))

In [14]:
model_normal.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Definir una estrategia distribuida

In [15]:
distribute = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)


### Definir un modelo distribuido

In [16]:
with distribute.scope():
  model_distributed = tf.keras.models.Sequential()
  model_distributed.add(tf.keras.layers.Dense(units=128, activation='relu', input_shape=(784,)))
  model_distributed.add(tf.keras.layers.Dropout(rate=0.2))
  model_distributed.add(tf.keras.layers.Dense(units=10, activation='softmax'))
  model_distributed.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['sparse_categorical_accuracy'])

### Comparar la velocidad de entrenamiento normal vs distribuida

In [19]:
start_time = time.time()
model_distributed.fit(X_train, y_train, epochs=10, batch_size=25)
print("El entrenamiento distribuido ha tardado: {}".format(time.time() - start_time))

Epoch 1/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.0414 - sparse_categorical_accuracy: 0.9859
Epoch 2/10
2400/2400 [==============================] - 6s 2ms/step - loss: 0.0389 - sparse_categorical_accuracy: 0.9868
Epoch 3/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.0360 - sparse_categorical_accuracy: 0.9875
Epoch 4/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.0350 - sparse_categorical_accuracy: 0.9874
Epoch 5/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.0331 - sparse_categorical_accuracy: 0.9890
Epoch 6/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.0330 - sparse_categorical_accuracy: 0.9887
Epoch 7/10
2400/2400 [==============================] - 6s 3ms/step - loss: 0.0294 - sparse_categorical_accuracy: 0.9898
Epoch 8/10
2400/2400 [==============================] - 6s 2ms/step - loss: 0.0290 - sparse_categorical_accuracy: 0.9902
Epoch 9/10
2400/2400 [==========

In [18]:
start_time = time.time()
model_normal.fit(X_train, y_train, epochs=10, batch_size=25)
print("El entrenamiento normal ha tardado: {}".format(time.time() - start_time))

Epoch 1/10
2400/2400 [==============================] - 5s 2ms/step - loss: 1.2674 - sparse_categorical_accuracy: 0.6142
Epoch 2/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.7037 - sparse_categorical_accuracy: 0.6923
Epoch 3/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.5707 - sparse_categorical_accuracy: 0.7580
Epoch 4/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.4803 - sparse_categorical_accuracy: 0.8065
Epoch 5/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.3769 - sparse_categorical_accuracy: 0.8970
Epoch 6/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.2372 - sparse_categorical_accuracy: 0.9604
Epoch 7/10
2400/2400 [==============================] - 5s 2ms/step - loss: 0.1800 - sparse_categorical_accuracy: 0.9663
Epoch 8/10
2400/2400 [==============================] - 4s 2ms/step - loss: 0.1498 - sparse_categorical_accuracy: 0.9696
Epoch 9/10
2400/2400 [==========